In [97]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
import json
import time
from datetime import datetime

#driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [116]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url ="https://www.bigkinds.or.kr/v2/news/index.do"
driver.get(url)
#ActionChains생성
action = ActionChains(driver)

driver.implicitly_wait(3)
element = driver.find_element(By.CLASS_NAME,"news-report-search-btn")
element.click()

In [136]:
def save (e, result_list, page):
    print(e)
    print(f"크롤링 종료:{datetime.now()}")
    print("page: "+str(page))
    print("마지막 뉴스: "+li[-1]["title"])
    with open('crawling_data_100_second.json', 'w', encoding='utf-8') as f:
          json.dump(result_list, f, indent="\t", ensure_ascii=False)
    

In [139]:
# 최신 뉴스가 계속 업데이트되기 때문에 중복데이터를 피하기위해 뒷페이지에서 앞페이지로 이동하면서 크롤링
# 1. 크롤링 원하는 페이지로 이동
# 2. page를 100개 씩 표시로 변경
# 3. code 실행
# 4. 실행 중간에 클릭된 뉴스가 안꺼지면 수동으로 클릭필요
# 5. 그 외 error발생 시 종료하고 크롤링된 data 저장

li = []

try:
    for p in range (1, 200):
        for i in range(1,101):
            try:
                # 하나의 뉴스 데이터를 담을 dic
                base_dic = {}
                title = driver.find_element(By.XPATH,f'//*[@id="news-results"]/div[{i}]/div/div[2]/a/div/strong/span')
                base_dic['title'] = title.text

                try:
                    news_agency = driver.find_element(By.XPATH,f'//*[@id="news-results"]/div[{i}]/div/div[2]/div/div/a')
                    base_dic['news_agency'] = news_agency.text
                    base_dic['origin_url'] = news_agency.get_attribute('href')
                except:
                    base_dic['news_agency'] = driver.find_element(By.XPATH,'//*[@id="news-results"]/div[1]/div/div[2]/div/div').text
                    base_dic['origin_url'] = ''
                action.move_to_element(news_agency).perform()
                time.sleep(0.5)

                title.click()
                time.sleep(1)


                #driver.implicitly_wait(5)
                base_dic['tag'] = driver.find_element(By.CLASS_NAME,'bullet-keyword').text
                base_dic['created_date'] = driver.find_element(By.XPATH,'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/div[1]/ul/li[1]').text
                base_dic['reporter'] = driver.find_element(By.XPATH,'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[1]/div[1]/ul/li[2]').text

                try:
                    base_dic['image'] = driver.find_element(By.XPATH,'//*[@id="news-detail-modal"]/div/div/div[1]/div/div[2]/div/img').get_attribute('src')
                except:
                    base_dic['image'] = ''

                base_dic['news_contents'] = driver.find_element(By.CLASS_NAME,'news-view-body').text

                quit = driver.find_element(By.XPATH,'//*[@id="news-detail-modal"]/div/div/div[2]/button')

                action.send_keys(Keys.PAGE_DOWN).perform()
                time.sleep(0.4)
                quit.click()
                time.sleep(1)
                li.append(base_dic)
            except:
                print("error")
                print(base_dic)
                pass
            
        next_page = driver.find_element(By.CLASS_NAME,'page-prev.page-link')
        action.move_to_element(next_page).perform()
        next_page.click()
        time.sleep(1)
        
except Exception as e:
    save(e, li, p)
    print(base_dic)


error
{'title': '건설 표준시장단가 1월보다 2.63% 상승', 'news_agency': '영남일보', 'origin_url': 'https://www.yeongnam.com/web/view.php?key=20230430010003802'}
error
{'title': '“이효리가 축가 불러줬다”…핑크 미니스커트 ‘축가’', 'news_agency': '서울신문', 'origin_url': 'http://www.seoul.co.kr/news/newsView.php?id=20230501500009'}
error
{'title': '수단 군벌, 72시간 휴전 재연장', 'news_agency': '매일신문 사회>사건_사고', 'origin_url': ''}
error
{'title': '서울 은평구도 무더기 전세사기…빌라 100채 "최소 200억원 피해"', 'news_agency': '매일신문', 'origin_url': 'https://news.imaeil.com/page/view/2023050108060798101'}
error
{'title': "경기도 '종합·개인지방소득세 확정신고 이달 말까지 하세요'", 'news_agency': '서울경제', 'origin_url': 'http://www.sedaily.com/NewsView/29PERGS9H0'}
error
{'title': '‘생성형 AI’ 이제 시작인데… “우리 데이터 쓰지 말라” 벌써 밥그릇 싸움[ICT]', 'news_agency': '문화일보', 'origin_url': 'http://www.munhwa.com/news/view.html?no=2023050101032007018001'}
error
{'title': '농번기 일손부족 공동급식으로 해결', 'news_agency': 'KBS', 'origin_url': 'https://news.kbs.co.kr/news/view.do?ncd=7664467&amp;ref=DA'}
error
{'title': '"7개월째 수출 감소"